# Bike Sharing Demand

In [63]:
import pandas
import numpy
import time
from datetime import datetime
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge, Lasso
import sklearn.model_selection as model_selection
import sklearn.metrics as metrics
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.cross_validation import KFold

### Data Loading
Load the training and testing data from the given CSV files

In [23]:
# X should not contain any of the label columns and y is simply the count
train_data = pandas.read_csv('data/train.csv')
X = train_data.drop(['count', 'casual', 'registered'], axis=1)
y = train_data['count']
y1 = train_data['casual']
y2 = train_data['registered']

test_data = pandas.read_csv('data/test.csv')

### Evaluation
The scoring function is the Root Mean Squared Logarithmic Error given by

$ \sqrt{\frac{1}{n} \sum_{i=1}^n (\log(p_i + 1) - \log(a_i+1))^2 } $

Where

* $n$ is the number of hours in the test set
* $pi$ is your predicted count
* $ai$ is the actual count
* $log(x)$ is the natural logarithm

In [24]:
def rmsle(y, y_):
    log1 = numpy.nan_to_num(numpy.array([numpy.log(v + 1) 
                                         for v 
                                         in y]))
    log2 = numpy.nan_to_num(numpy.array([numpy.log(v + 1) 
                                         for v 
                                         in y_]))
    calc = (log1 - log2) ** 2
    return numpy.sqrt(numpy.mean(calc))

# create a custom scorer to be used in grid search, etc
scorer = metrics.make_scorer(score_func=rmsle, 
                             greater_is_better=False)

### Submission
In order to submit to Kaggle we have to generate predictions from the test set and output them to a file with the following format

~~~~
datetime,count
2011-01-20 00:00:00,0
2011-01-20 01:00:00,0
2011-01-20 02:00:00,0
...
...
~~~~

In [25]:
def generate_kaggle_submission(
        transformer, 
        regressor, 
        X_train, 
        y_train,
        test_data):
    
    # train the final model on the transformed data
    regressor.fit(transformer(X_train), 
                  y_train)

    # create a dataframe containing the datetimes to predict and then
    # add the predictions from the trained pipeline
    predictions = pandas.DataFrame(test_data['datetime'])
    predictions['count'] = regressor.predict(transformer(test_data)).astype('int')

    # create a submission file from the result tagged with the current time
    predictions.to_csv('submissions/submission{0}.csv'.format(str(int(time.time()))), 
                       sep=',', 
                       index=False)

### Partition Data

Split out the given training data into a train and a test set and use all of the available parameters

In [26]:
def partition_train_test(
        X, 
        y,
        y1,
        y2,
        split_percentage = .8):
    mask = numpy.random.rand(len(X)) < split_percentage
    X_train = X[mask]
    y_train = y[mask]
    y1_train = y1[mask]
    y2_train = y2[mask]
    X_test = X[~mask]
    y_test = y[~mask]
    y1_test = y1[~mask]
    y2_test = y2[~mask]
    
    print('{0} training examples and {1} testing examples'.format(len(X_train), 
                                                                  len(X_test)))
    
    return X_train, X_test, y_train, y_test, y1_train, y1_test, y2_train, y2_test

X_train, X_dev, y_train, y_dev, y1_train, y1_dev, y2_train, y2_dev = partition_train_test(X, y, y1, y2)

8747 training examples and 2139 testing examples


### Initial Feature Engineering

All of the data is already numeric except for datetime. Replace the datetime with distinct numeric parameters for hour, day, month and year. Then display the summary of the data as a sanity check.

In [27]:
def get_date(my_datetime):
    return datetime.strptime(
        my_datetime, 
        '%Y-%m-%d %H:%M:%S')

def simple_feature_eng(data):
    copy = data.copy()
    copy['hour'] = copy.datetime.apply(lambda x: x.split()[1].split(':')[0]).astype('int')
    copy['day'] = copy.datetime.apply(lambda x: x.split()[0].split('-')[2]).astype('int')
    copy['month'] = copy.datetime.apply(lambda x: x.split()[0].split('-')[1]).astype('int')
    copy['year'] = copy.datetime.apply(lambda x: x.split()[0].split('-')[0]).astype('int')
    copy = copy.drop(['datetime'], axis=1)
    return copy

print(simple_feature_eng(X).describe())

             season       holiday    workingday       weather         temp  \
count  10886.000000  10886.000000  10886.000000  10886.000000  10886.00000   
mean       2.506614      0.028569      0.680875      1.418427     20.23086   
std        1.116174      0.166599      0.466159      0.633839      7.79159   
min        1.000000      0.000000      0.000000      1.000000      0.82000   
25%        2.000000      0.000000      0.000000      1.000000     13.94000   
50%        3.000000      0.000000      1.000000      1.000000     20.50000   
75%        4.000000      0.000000      1.000000      2.000000     26.24000   
max        4.000000      1.000000      1.000000      4.000000     41.00000   

              atemp      humidity     windspeed          hour           day  \
count  10886.000000  10886.000000  10886.000000  10886.000000  10886.000000   
mean      23.655084     61.886460     12.799395     11.541613      9.992559   
std        8.474601     19.245033      8.164537      6.91583

### Initial Regressor

Decision Tree ensembles, particularly Boosted Decision Trees, have fairly good performance over a wide variety of use cases as demonstrated [here](https://ucb-mids.s3.amazonaws.com/prod/DATASCI+W207+Intro+to+Machine+Learning/Readings/caruana.icml06.pdf). Since the values for count have to be both non-negative and an integer we will subclass the Gradient Boosting Regressor to force the predictions to accomidate that requirement.

In [28]:
class PositiveIntegerGradientBoostingRegressor(GradientBoostingRegressor):
    def predict(
            self, 
            X):
        prediction = super(
            PositiveIntegerGradientBoostingRegressor, 
            self).predict(X)
        return numpy.around(prediction.clip(0))

### Model Fit

We will use GridSearch to tune over a range values of max depth for a Gradient Descent Boosted Decision Tree regressor preceeded by our initial feature engineering transformer. We will also use the given RMSLE error function as a custom scoring function. Fit the model and evaluate the resulting predictions on the held out dev data set.

In [78]:
def simple_grid_search(
        regressor, 
        transformer,
        param_grid, 
        X_train, 
        y_train, 
        X_dev, 
        y_dev):
    pipeline = Pipeline([('reg', regressor)])

    model = model_selection.GridSearchCV(pipeline, 
                                         param_grid, 
                                         scorer,
                                         n_jobs=-1)
    
    transformed_X_train = transformer(X_train)
    transformed_X_dev = transformer(X_dev)
    model.fit(transformed_X_train,
              y_train)
    
    print('Best Parameters: {0}'.format(model.best_params_))
    print('RMSLE: {0}'.format(rmsle(y_dev, 
                                    model.predict(transformed_X_dev))))
    return model.best_params_


In [68]:
hyperparameters = simple_grid_search(
    PositiveIntegerGradientBoostingRegressor(n_estimators=100), 
    simple_feature_eng,
    [{'reg__max_depth': list(range(3, 12))}],
    X_train,
    y_train,
    X_dev,
    y_dev)

Transforming dataset with 9 features
Fitting model with 12 features
Best Parameters: {'reg__max_depth': 11}
RMSLE: 0.3573104315806729


##### Initial Submission Generation
Generate the first submission to Kaggle. This resulted in a score of **.50555**.

In [13]:
# best hyperparameters found from simple_grid_search above
max_depth = 11

generate_kaggle_submission(simple_feature_eng,
                           PositiveIntegerGradientBoostingRegressor(
                               n_estimators=1000, 
                               max_depth=max_depth),
                           X,
                           y,
                           test_data)

### Exploratory data analysis
#### 1. Hourly trend of bike demand 
The figure below shows the trend of bike demand aggregated hourly for the traininig dataset. The trend highlights the gradual increase in demand as we apporach morning office hours, tapering off during the day time. the demand again picks up in after office hours.
<img src="HourlyBikeCountTrend.png" style="width: 600px;"/>

#### 2. Hourly trend of Registered and Casual bike demand
The figure below shows the trend of bike demand aggregated hourly for Casual and Registered bike users over the entire traininig dataset. The figure highlights the differences of trends observed for these two groups. The registered users have similar trend menioned in point 1 above. The demand for casual users peaks in the office hours and trails down during the non office hours. 
<img src="HourlyBikeCountTrendRegisteredVsCasual.png" style="width: 600px;"/>

#### 3. Weekday trend of Registered and Casual bike demand
The figure below shows the demand curve for Casual and Registered users for day of a week aggregated for the training data set. The demand for casual users is high on weekends while that of registered users is high during weekdays.
<img src="WkdayBikeCountTrendRegisteredVsCasual.png" style="width: 500px;"/>

### 4. Impact of working and non-working day on demand from registered and casual bikers
The trend below highlghts the differences on the bike demand based on working day and non working day. Casual and registered bike users exhibit similar characteristics on non working day.
<img src="WorkNonworkBikeCountTrendRegisteredVsCasual.png" style="width: 1000px;"/>


## Observations
Based on the above exploratory analysis and visual inspection of the data we have made the following observations:

1. The behavior of the registered bikers is very different from the behavior of the casual bikers in terms of time of day, date of week, etc.
2. The training set of data consists of only the first 20 days of any given month and the testing data always consists of the last 7-11 days.
3. The mean values and standard deviations vary widely among different parameters.
4. Generally speaking model displays some signs of overfitting given the training RMSE of .35 and the testing RMSE of over .5. 

## Plan
1. To account for the difference in behavior between the two groups we will construct 2 completely distinct models and sum the resulting predictions to get the overall demand. In this way we can account for the disparate behavior between the two customer types.
2. We will create an ensemble of sorts by using a purely time-series method to forecast the demand on the last 7-11 days of each month. Because the monthly data itself is stationary and fairly periodic this should give us a reasonable (on the average) estimate of the remaining days demand each month. This data will then become part of training data and hopefully will alleviate some of the inaccuracies caused by prediction from ranges of parameters not in the training set.
3. We will simply standardize all of the features to zero mean and unit variance.
4. We may try choosing some other core model than Boosted Decision Trees even though this variant of Decision Trees (limiting depth, iterating over multiple trees) generally limits overfitting. A simple model that is less inclined to overfit such as Logistic Regression may work better.

As an overall note dependant variables tend to have outliers. So we will consider removing outliers and/or predicting a transformed version of the output.

## Step 1: Casual/Registered Model Split
Working from part one of the plan we will simply break the prediction down into two models: one predicting the number of registered users in a day and the other predicting the number of casual users. In addition we will take a simple log(x+1) transform on the regressor to normalize for outliers, and reverse the transform for the submission.

In [71]:
casual_hyperparameters = simple_grid_search(
    GradientBoostingRegressor(n_estimators=100), 
    simple_feature_eng,
    [{'reg__max_depth': list(range(3, 12))}],
    X_train,
    numpy.log1p(y1_train + 1),
    X_dev,
    numpy.log1p(y1_dev + 1))

registered_hyperparameters = simple_grid_search(
    GradientBoostingRegressor(n_estimators=100), 
    simple_feature_eng,
    [{'reg__max_depth': list(range(3, 12))}],
    X_train,
    numpy.log1p(y2_train + 1),
    X_dev,
    numpy.log1p(y2_dev + 1))

Transforming dataset with 9 features
Fitting model with 12 features
Best Parameters: {'reg__max_depth': 6}
RMSLE: 0.13698979192894803
Transforming dataset with 9 features
Fitting model with 12 features
Best Parameters: {'reg__max_depth': 4}
RMSLE: 0.07704778792073358


#####  Second Submission Generation
Generate our second submission to Kaggle for a score. This requires and input-output transformation of the count variable and summing the resulting generation of scores from the casual and registered users. This resulted in a submission score of **.46154** which was a vast improvement over our previous score.

In [73]:
# best hyperparameters found from simple_grid_search above
casual_max_depth = 6
registered_max_depth = 4

casual_regressor = GradientBoostingRegressor(n_estimators=1000, 
                                             max_depth=casual_max_depth)
registered_regressor = GradientBoostingRegressor(n_estimators=1000, 
                                                 max_depth=registered_max_depth)

# train the final models on the transformed data and the transformed labels
casual_regressor.fit(simple_feature_eng(X), 
                     numpy.log1p(y1 + 1))
registered_regressor.fit(simple_feature_eng(X), 
                         numpy.log1p(y2 + 1))

# create a dataframe containing the datetimes to predict
predictions = pandas.DataFrame(test_data['datetime'])

# predict for both the casual and registered users
casual_predictions = casual_regressor.predict(simple_feature_eng(test_data))
registered_predictions = registered_regressor.predict(simple_feature_eng(test_data))

# reverse the transform for both predictions and then add the result
predictions['count'] = (numpy.exp(casual_predictions) - 1 + 
                        numpy.exp(registered_predictions) - 1).astype('int')

# create a submission file from the result tagged with the current time
predictions.to_csv('submissions/submission{0}.csv'.format(str(int(time.time()))), 
                   sep=',', 
                   index=False)

## Part 2: Model Selection
In addition to splitting out the models as done above, we would like to perform a search over different types of models as well as their hyperparameters. The above improvement was primarily due to the log transform of the output data which indicates that the model is somewhat sensitive to outliers. In addition some simple feature engineering changes (not shown in this notebook) improved the score on the training set but worsened the score on the test set. These two phenomenon are classic indicators of model overfitting. In order to combat that overfitting we would like to try a variety of different models. In addition there is some [evidence](http://www.jmlr.org/papers/volume11/cawley10a/cawley10a.pdf) that simple cross-validation for both model selection and performance estimation can result in further overfitting. Thus we will use nested cross-validation to select a model for the registered and casual users.

In [64]:
def nested_cross_validation(
        models, 
        params, 
        transformer,
        X, 
        y):
    transformed_X = transformer(X)
    scores = [[] for _ in range(len(models))]
    
    for tr, ts in KFold(len(transformed_X)):
        for i, (model, param) in enumerate(zip(models, params)):
            model = model_selection.GridSearchCV(
                model, 
                param, 
                scorer,
                n_jobs=4)
            model.fit(transformed_X.iloc[tr], 
                      y[tr])
            scores[i].append(rmsle(y[ts], 
                                   model.predict(transformed_X.iloc[ts])))
    return zip(models, np.mean(scores, 1))
    

regressors = [SVR(), 
              RandomForestRegressor(),
              GradientBoostingRegressor(),
              Ridge(),
              Lasso()]
parameters = [{'C': [0.01, 0.05, 0.1, .5, 1]}, 
              {'max_depth': list(range(3, 12))},
              {'max_depth': list(range(3, 12))},
              {'alpha': [0.01, 0.05, 0.1, .5, 1]},
              {'alpha': [100, 20, 10, 2, 1]}]

casual_results = nested_cross_validation(
                    regressors,
                    parameters,
                    simple_feature_eng,
                    X,
                    numpy.log1p(y1 + 1))

print('{:<30} {:<15}'.format('Model (casual)', 'RMSLE'))
print('-------------------------------------')
for i in casual_results:
    print('{:<30} {:<15,.4f}'.format(i[0].__class__.__name__, 
                                               i[1]))
    
registered_results = nested_cross_validation(
                        regressors,
                        parameters,
                        simple_feature_eng,
                        X,
                        numpy.log1p(y2 + 1))

print('\n')
print('{:<30} {:<15}'.format('Model (registered)', 'RMSLE'))
print('-------------------------------------')
for i in registered_results:
    print('{:<30} {:<15,.4f}'.format(i[0].__class__.__name__, 
                                               i[1]))

Model (casual)                 RMSE           
-------------------------------------
SVR                            0.3398         
RandomForestRegressor          0.1639         
GradientBoostingRegressor      0.1571         
Ridge                          0.2590         
Lasso                          0.2802         
Model (registered)             RMSE           
-------------------------------------
SVR                            0.2712         
RandomForestRegressor          0.1173         
GradientBoostingRegressor      0.1034         
Ridge                          0.2223         
Lasso                          0.2415         


For both casual and registered users Random Forrest and Gradient Descent Boosted Decision Trees produce superior results as compared to any other regression model. Since they are both very similar in terms of their RMSLE we will create a simple ensemble model that averages the output of the two models.

To be more thorough with this iteration we perform a grid search over all relevant parameters rather than arbitrarily setting n_estimators in the GradientBoostingRegressor as in previous iterations.

In [79]:
print('Casual user GradientBoostingRegressor')
casual_gb_hyperparameters = simple_grid_search(
    GradientBoostingRegressor(), 
    simple_feature_eng,
    {'reg__max_depth': list(range(3, 12)),
     'reg__n_estimators': [100, 200, 300, 400, 500, 1000]},
    X_train,
    numpy.log1p(y1_train + 1),
    X_dev,
    numpy.log1p(y1_dev + 1))

print('\n')
print('Registered user GradientBoostingRegressor')
registered_gb_hyperparameters = simple_grid_search(
    GradientBoostingRegressor(), 
    simple_feature_eng,
    {'reg__max_depth': list(range(3, 12)),
     'reg__n_estimators': [100, 200, 300, 400, 500, 1000]},
    X_train,
    numpy.log1p(y2_train + 1),
    X_dev,
    numpy.log1p(y2_dev + 1))

Casual user GradientBoostingRegressor
Best Parameters: {'reg__max_depth': 6, 'reg__n_estimators': 100}
RMSLE: 0.13703367972045005


Registered user GradientBoostingRegressor
Best Parameters: {'reg__max_depth': 3, 'reg__n_estimators': 300}
RMSLE: 0.0747842138473513


In [80]:
print('Casual user RandomForestRegressor')
casual_rf_hyperparameters = simple_grid_search(
    RandomForestRegressor(), 
    simple_feature_eng,
    {'reg__min_samples_split': list(range(3, 12)),
     'reg__n_estimators': [100, 500, 1000, 2000, 5000]},
    X_train,
    numpy.log1p(y1_train + 1),
    X_dev,
    numpy.log1p(y1_dev + 1))

print('Registered user RandomForestRegressor')
registered_rf_hyperparameters = simple_grid_search(
    RandomForestRegressor(), 
    simple_feature_eng,
    {'reg__min_samples_split': list(range(3, 12)),
     'reg__n_estimators': [100, 500, 1000, 2000, 5000]},
    X_train,
    numpy.log1p(y2_train + 1),
    X_dev,
    numpy.log1p(y2_dev + 1))

Casual user RandomForestRegressor
Best Parameters: {'reg__min_samples_split': 11, 'reg__n_estimators': 2000}
RMSLE: 0.13989044089880226
Registered user RandomForestRegressor
Best Parameters: {'reg__min_samples_split': 8, 'reg__n_estimators': 500}
RMSLE: 0.0769744595514548


#####  Third Submission Generation
Generate our third submission to Kaggle for a score. This requires training 4 models: 2 for the casual users and 2 for the registered users. These models each use the optimal hyperparameters found above. The results of each casual and registered model are averaged together, and then summed to get a final score for 'count'. This resulted in a submission score of ***.44112***.

In [83]:
# best hyperparameters found from simple_grid_search above
casual_gb_max_depth = 6
casual_gb_n_estimators = 100
registered_gb_max_depth = 3
registered_gb_n_estimators = 300

casual_rf_min_samples_split = 11
casual_rf_n_estimators = 2000
registered_rf_min_samples_split = 8
registered_rf_n_estimators = 500

casual_gb_regressor = GradientBoostingRegressor(n_estimators=casual_gb_n_estimators, 
                                                max_depth=casual_gb_max_depth)
registered_gb_regressor = GradientBoostingRegressor(n_estimators=registered_gb_n_estimators, 
                                                    max_depth=registered_gb_max_depth)
casual_rf_regressor = RandomForestRegressor(min_samples_split=casual_rf_min_samples_split,
                                             n_estimators=casual_rf_n_estimators)
registered_rf_regressor = RandomForestRegressor(min_samples_split=registered_rf_min_samples_split,
                                                 n_estimators=registered_rf_n_estimators)

# train the final models on the transformed data and the transformed labels
casual_gb_regressor.fit(simple_feature_eng(X), 
                        numpy.log1p(y1 + 1))
casual_rf_regressor.fit(simple_feature_eng(X), 
                        numpy.log1p(y1 + 1))
registered_gb_regressor.fit(simple_feature_eng(X), 
                            numpy.log1p(y2 + 1))
registered_rf_regressor.fit(simple_feature_eng(X), 
                            numpy.log1p(y2 + 1))

# create a dataframe containing the datetimes to predict
predictions = pandas.DataFrame(test_data['datetime'])

# predict for both the casual and registered users and reverse the transformation
casual_gb_predictions = numpy.exp(casual_gb_regressor.predict(simple_feature_eng(test_data))) - 1
casual_rf_predictions = numpy.exp(casual_rf_regressor.predict(simple_feature_eng(test_data))) - 1
registered_gb_predictions = numpy.exp(registered_gb_regressor.predict(simple_feature_eng(test_data))) - 1
registered_rf_predictions = numpy.exp(registered_rf_regressor.predict(simple_feature_eng(test_data))) - 1

# the casual and registered predictions are half of the sum from the two models
casual_predictions = (casual_gb_predictions + casual_rf_predictions) / 2.0
registered_predictions = (registered_gb_predictions + registered_rf_predictions) / 2.0

# add the casual and registered predictions and convert to an integer
predictions['count'] = (casual_predictions + registered_predictions).astype('int')

# create a submission file from the result tagged with the current time
predictions.to_csv('submissions/submission{0}.csv'.format(str(int(time.time()))), 
                   sep=',', 
                   index=False)

## Part 3: Improved Feature Engineering
Possible Additions
* Normalize parameters
* Remove outliers
* Add categorical parameters
* Create bins for some parameters (hour, windspeed)

## Timeseries Forecast